# Detecting Deep Pain with Deep Learning

***Ruslan Klymentiev***

***created: 2018-05-03***

<center><img src="http://www.arksurgicalhospital.com/wp-content/uploads/2016/07/severe-spine-pain-spine-surgery-Arkansas.-Arkansas-Surgical-Hospital.jpg" width="400"></center>

- <a href='#over'>Overview of the project</a>  
- <a href='#setup'>Setting up the environment and data import</a>
- <a href='#clean'>Data Cleaning</a>
- <a href='#eda'>Exploratory Data Analysis</a>
- <a href='#keras'>Building Keras Model</a>
    - <a href='#tune1'>Tuning the Model (1)</a>
    - <a href='#tune2'>Tuning the Model (2)</a>
    - <a href='#sum'>Model Comparison</a>
 

## <a id='over'>Overview of the project</a>  

Today I am going to use Keras for classification problem on dataset which contains data about abnormal or normal spine conditions using collected physical spine details/data.

## <a id='setup'>Setting up the environment and data import</a>  

In [ ]:
import numpy as np 
import pandas as pd 
import os
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
np.random.seed(123)
Data = pd.read_csv('../input/Dataset_spine.csv')

In [ ]:
Data.sample(5)

## <a id='clean'>Data Cleaning</a>

Thanks to [AnthonyRidding kernel](https://www.kaggle.com/anfro18/lower-back-pain-algorithm) I found a lables to replace column names in a dataframe. Also have changed the ouput values:

**Normal** condition -> **0**

**Abnormal** condition -> **1**

In [ ]:
Data = Data.drop('Unnamed: 13', axis=1) #drop the last column
Data.columns = ['Pelvic Incidence','Pelvic Tilt','Lumbar Lordosis Angle','Sacral Slope','Pelvic Radius', 'Spondylolisthesis Degree', 'Pelvic Slope', 'Direct Tilt', 'Thoracic Slope', 'Cervical Tilt','Sacrum Angle', 'Scoliosis Slope','Outcome'] #thanks to AnthonyRidding
Data = Data.sample(Data.shape[0]) #shuffle the data since right now it is sorted by outcome column
Data.Outcome = Data.Outcome.replace('Normal', 0).replace('Abnormal', 1).apply(int)
Data.sample(5)

Looks much better now. Checking for missing values in a dataset:

In [ ]:
Data.isnull().sum()

Yay, no missing values. So the next step is

## <a id='eda'>Exploratory Data Analysis</a>

In [ ]:
scatter_matrix(Data, alpha=0.2, figsize=(30, 30))
plt.show()

There is some strange outlier in **Spondylolisthesis Degree** column, so I am going to drop the row with that value.

In [ ]:
Data = Data[Data['Spondylolisthesis Degree'] != Data['Spondylolisthesis Degree'].max()]

correlations = Data.corr()
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)
cax = ax.matshow(correlations, vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,Data.shape[1],1)
ax.set_xticks(ticks)
ax.set_yticks(ticks)
plt.xticks(rotation=90)
ax.set_xticklabels(Data.columns.values.tolist())
ax.set_yticklabels(Data.columns.values.tolist())
plt.show()

## <a id='keras'>Building Keras Model</a>  

> Keras is a high-level neural networks API, written in Python and capable of running on top of *TensorFlow, CNTK*, or *Theano*. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

In [ ]:
#split data to Target and Predictiors arrays
train, test = train_test_split(Data, test_size=0.25)
X_train = np.array(train.drop('Outcome', axis=1))
y_train = to_categorical(train.Outcome)
X_test = np.array(test.drop('Outcome', axis=1))
y_test = to_categorical(test.Outcome)
n_cols = X_train.shape[1]

In [ ]:
#building a simple model with 2 hidden layers and 1 output layer.
model_0 = Sequential()
model_0.add(Dense(300, activation='relu', input_dim=n_cols))
model_0.add(Dense(200, activation='relu'))
model_0.add(Dense(2, activation='softmax'))
model_0.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_0 = model_0.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=False)

In [ ]:
#plots of the scores
plt.figure(figsize=(15,6))
plt.style.use('bmh')
plt.subplot(1, 2, 1)
plt.plot(history_0.history['val_loss'], 'b')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history_0.history['val_acc'], 'b')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()

So far model predicts on validation set with ~80% accuracy.

### <a id='tune1'>Tuning the Model (1)</a>  

Adding more layers and neurons:

In [ ]:
#stop optimization when the validation loss hasn't improved for 10 epochs
#early_stopping_monitor = EarlyStopping(patience=10) 

model_1 = Sequential()
model_1.add(Dense(600, activation='relu', input_dim=n_cols))
model_1.add(Dense(300, activation='relu'))
model_1.add(Dense(300, activation='relu'))
model_1.add(Dense(300, activation='relu'))
model_1.add(Dense(2, activation='softmax'))
model_1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_1 = model_1.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=False)

plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
plt.plot(history_1.history['val_loss'], 'g')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history_1.history['val_acc'], 'g')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()

### <a id='tune2'>Tuning the Model (2)</a>

Adding Dropout (0.2 ratio)

> *Dropout is a regularization technique for deep neural networks introduced by Hinton which consists of preventing co-adaptation of feature detectors by randomly turning oﬀ a portion of neurons at every training iteration but using the entire network (with weights scaled down) at test time.*

In [ ]:
model_2 = Sequential()
model_2.add(Dense(600, activation='relu', input_dim=n_cols))
model_2.add(Dropout(0.2))
model_2.add(Dense(300, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(Dense(300, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(Dense(300, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(Dense(2, activation='softmax'))
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history_2 = model_2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, verbose=False)

plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
plt.plot(history_2.history['val_loss'], 'r')
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history_2.history['val_acc'], 'r')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.show()

### <a id='sum'>Model Comparison</a>

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(1, 2, 1)
plt.plot(history_0.history['val_loss'], 'b', linestyle = ':', linewidth=1, alpha = 0.5)
plt.plot(history_1.history['val_loss'], 'g', linestyle = ':', linewidth=1, alpha = 0.5)
plt.plot(history_2.history['val_loss'], 'r', linewidth=1, alpha = 0.8)
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')

plt.subplot(1, 2, 2)
plt.plot(history_0.history['val_acc'], 'b', linestyle = ':', linewidth=1, alpha = 0.5)
plt.plot(history_1.history['val_acc'], 'g', linestyle = ':', linewidth=1, alpha = 0.5)
plt.plot(history_2.history['val_acc'], 'r', linewidth=1, alpha = 0.8)
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.savefig('graph.png', dpi=50)
plt.legend(handles=[mpatches.Patch(color='r', label='Model 2 (Last one)'),
                    mpatches.Patch(color='g', label='Model 1'),
                    mpatches.Patch(color='b', label='Model 0')])
plt.show()

In [ ]:
y_test = np.argmax(y_test,axis = 1) 

In [ ]:
def check_model(mod):
    y = mod.predict(X_test)
    y = np.argmax(y, axis = 1) 
    cm = confusion_matrix(y_test, y)
    acc = (cm[0][0] + cm[1][1]) / (cm[0][0] + cm[1][1] + cm[1][0] + cm[1][0])
    return y, round(acc, 2)

y_pred_0, acc_0 = check_model(model_0)
y_pred_1, acc_1 = check_model(model_1)
y_pred_2, acc_2 = check_model(model_2)

print("Accuracy for Model 0 (blue): ", acc_0)
print("Accuracy for Model 1 (green): ", acc_1)
print("Accuracy for Model 2 (red): ", acc_2)

In [ ]:
fpr_0, tpr_0, _ = roc_curve(y_test, y_pred_0)
fpr_1, tpr_1, _ = roc_curve(y_test, y_pred_1)
fpr_2, tpr_2, _ = roc_curve(y_test, y_pred_2)

plt.figure(figsize=(10,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_0, tpr_0, c='b', label='Model 0')
plt.plot(fpr_1, tpr_1, c='g', label='Model 1')
plt.plot(fpr_2, tpr_2, c='r', label='Model 2')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#caclulation AUC (area under the curve)
print('AUC for Model 0 (blue): ', round(roc_auc_score(y_test, y_pred_0), 2))
print('AUC for Model 1 (green): ', round(roc_auc_score(y_test, y_pred_1), 2))
print('AUC for Model 2 (red): ', round(roc_auc_score(y_test, y_pred_2), 2))